In [55]:
import pandas as pd
import numpy as np

In [56]:
df = pd.read_csv("./UH_2023/UH_2023_TRAIN.txt", sep = "|")

In [57]:
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str})

In [58]:
def fun(x, mode): 
    altitud = str(x).split('-')
    if altitud[0] == "nan":
        if mode == "nan_1":
            return -1
        return np.nan
    if mode == "nan_1":
            return x
    if mode == "min":
        return int(altitud[0])
    if mode == "diff":
        if len(altitud) == 1:
            return 0
        else:
            return int(altitud[1])-int(altitud[0])

df["ALTITUD_MIN"] = df["ALTITUD"].apply(fun, args=("min",))
df["ALTITUD_DIF"] = df["ALTITUD"].apply(fun, args=("diff",))
df["ALTITUD"] = df["ALTITUD"].apply(fun, args=("nan_1",))

In [59]:
df_estacion_gb_mean = df.groupby(by="ID_ESTACION").mean()
df_estacion_gb_mode = df.groupby(by="ID_ESTACION").agg(pd.Series.mode)

C:\Users\xavid\anaconda3\lib\site-packages\pandas\core\algorithms.py:968: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  warn(f"Unable to sort modes: {err}")
C:\Users\xavid\AppData\Local\Temp\ipykernel_10484\3219566741.py:2: FutureWarning: ['SUPERFICIE', 'PRODUCCION'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df_estacion_gb_mode = df.groupby(by="ID_ESTACION").agg(pd.Series.mode)


In [60]:
df_estacion_gb_mean.reset_index(inplace=True)
df_estacion_gb_mode.reset_index(inplace=True)

In [61]:
df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == "0", "ALTITUD"] = 390
df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == "4", "ALTITUD"] = 520

In [62]:
for i in df["ID_ESTACION"].unique():
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD_MIN"] = df_estacion_gb_mean.loc[df_estacion_gb_mean["ID_ESTACION"] == i,"ALTITUD_MIN"].values[0]
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD_DIF"] = df_estacion_gb_mean.loc[df_estacion_gb_mean["ID_ESTACION"] == i,"ALTITUD_DIF"].values[0]
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD"] = df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == i,"ALTITUD"].values[0]

In [63]:
df

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
0,14,76953,515,4,660,26,2,0,1,0.0000,22215.0,660.000000,0.00000
1,14,84318,515,4,660,26,2,0,1,0.0000,22215.0,660.000000,0.00000
2,14,85579,340,4,520,32,2,0,1,0.0000,20978.0,520.000000,0.00000
3,14,69671,340,4,520,32,2,0,1,0.0000,40722.0,520.000000,0.00000
4,14,14001,852,14,650-660,81,1,0,1,0.0000,14126.0,654.501718,9.19244
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9596,22,37461,239,6,700,52,2,0,1,3.6800,NaN,700.000000,0.00000
9597,22,58769,239,6,700,32,2,0,1,4.2500,NaN,700.000000,0.00000
9598,22,58769,239,6,700,59,2,0,1,4.0700,NaN,700.000000,0.00000
9599,22,88928,239,6,700,40,2,0,1,4.5727,NaN,700.000000,0.00000


In [64]:
import category_encoders as ce

encoder = ce.OneHotEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD"])

df_encoded = encoder.fit_transform(df)

In [65]:
df_encoded.head()

,CAMPAÑA,ID_FINCA_1,ID_FINCA_2,ID_FINCA_3,ID_FINCA_4,ID_FINCA_5,ID_FINCA_6,ID_FINCA_7,ID_FINCA_8,ID_FINCA_9,...,VARIEDAD_23,VARIEDAD_24,VARIEDAD_25,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
0,14,1,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,0.0,22215.0,660.000000,0.00000
1,14,0,1,0,0,0,0,0,0,0,...,0,0,0,2,0,1,0.0,22215.0,660.000000,0.00000
2,14,0,0,1,0,0,0,0,0,0,...,0,0,0,2,0,1,0.0,20978.0,520.000000,0.00000
3,14,0,0,0,1,0,0,0,0,0,...,0,0,0,2,0,1,0.0,40722.0,520.000000,0.00000
4,14,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0.0,14126.0,654.501718,9.19244


In [92]:
data = df_encoded.drop(index=df_encoded[df_encoded["CAMPAÑA"] == "22"].index)
to_train = data[(data["CAMPAÑA"] == "20") | (data["CAMPAÑA"] == "21")]
to_predict = data[(data["CAMPAÑA"] != "20") & (data["CAMPAÑA"] != "21")]

In [93]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor

X = to_train.drop(axis = 1, columns = ["SUPERFICIE"])
y = to_train["SUPERFICIE"]
model = {"model_name": "SVC", "model": MLPRegressor(), "params": {'hidden_layer_sizes':[(100),(100,100,100)],
                                                                  'learning_rate':["constant", "adaptive"],}}

grid_search = GridSearchCV(model["model"], model["params"], scoring="r2")
grid_search.fit(X, y)

GridSearchCV(estimator=MLPRegressor(),
             param_grid={'hidden_layer_sizes': [100, (100, 100, 100)],
                         'learning_rate': ['constant', 'adaptive']},
             scoring='r2')

In [94]:
print(grid_search.best_params_)
df = pd.DataFrame(grid_search.cv_results_)
df

{'hidden_layer_sizes': 100, 'learning_rate': 'adaptive'}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,14.371279,7.202682,0.098943,0.004439,100,constant,"{'hidden_layer_sizes': 100, 'learning_rate': '...",0.123603,0.591749,0.257049,0.374899,0.280996,0.325659,0.155407,3
1,14.513178,5.896693,0.100253,0.005974,100,adaptive,"{'hidden_layer_sizes': 100, 'learning_rate': '...",0.461397,0.246296,0.530267,0.561855,0.642507,0.488464,0.134343,1
2,7.712112,1.254087,0.107433,0.006283,"(100, 100, 100)",constant,"{'hidden_layer_sizes': (100, 100, 100), 'learn...",0.458992,0.465680,0.558451,0.441232,-2.256355,-0.066400,1.095735,4
3,9.669366,2.644009,0.105158,0.006142,"(100, 100, 100)",adaptive,"{'hidden_layer_sizes': (100, 100, 100), 'learn...",0.421468,0.514944,0.526402,0.391756,0.449338,0.460782,0.052308,2


In [98]:
modelMLPR = MLPRegressor(hidden_layer_sizes=(100), learning_rate="adaptive")
modelMLPR.fit(X, y)
to_predict = to_predict.drop(axis = 1, columns = ["SUPERFICIE"])
predicted = modelMLPR.predict(to_predict)

C:\Users\xavid\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
